# あすかの

## Database

DBname : Asuka

## Scraping


- Links
  - month : YYmm
  - url : []

In [1]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import re

In [2]:
client = MongoClient()
db = client['Asuka']
col = db.Links

In [3]:
headers ={'User-Agent': 'sample Vivaldi'}
links = ['http://blog.nogizaka46.com/asuka.saito/?d=201605']

In [4]:
def add_link(text):
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find(id='calendarPN')
    prev = tag.a.get('href')
    if prev[-4:] == '1111':
        col.insert_one({'month': prev[-4:], 'day_url': []})
        add_day_link(text, prev[-4:])
        return 
    else:
        links.append(prev)

def add_day_link(text, month):
    day_add = col.find_one({'month': month})
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find(id='daytable')
    for day_link in tag.find_all('a'):
        day_add['day_url'].append(day_link.get('href'))
        col.save(day_add)

In [5]:
while links:
    url = links.pop()
    month = url[-4:]
    col.insert_one({'month': month, 'day_url': []})
    r = requests.get(url, headers=headers)
    text = r.text
    add_day_link(text, month)
    add_link(text)

/home/kzkntt/.pyenv/versions/2.7.11/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead


## Words
- url_words
  - url : uri
  - text : 'text text text'
  - words : [text, text, text]

In [6]:
col2 = db.url_words

In [7]:
def union(p, q):
    for e in q:
        if e not in p:
            p.append(e)

In [8]:
all_day_url = []
for a in col.find({'month': re.compile(r'.*')}):
    union(all_day_url, a['day_url'])
    
all_day_url

[u'http://blog.nogizaka46.com/asuka.saito/?d=20160505',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160531',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160408',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160326',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160228',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160101',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20160124',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20151205',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20151105',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20151020',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150923',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150810',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150710',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150729',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150731',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150611',
 u'http://blog.nogizaka46.com/asuka.saito/?d=20150501',
 u'http://blog.nogizaka46.com/asuka.saito/?d=201

In [9]:
while all_day_url:
    url = all_day_url.pop()
    col2.insert_one({'url': url, 'text': '', 'words': []})
    

## MeCab